## Collision

Notebook for collision modeling in ASPECT 2026

In [ ]:
import os, sys, shutil, math
import numpy as np
from pathlib import Path
from shutil import rmtree, copy
from matplotlib import pyplot as plt
from matplotlib import gridspec, cm
from PIL import Image, ImageDraw, ImageFont
from scipy.interpolate import interp1d
from scipy.interpolate import UnivariateSpline
import datetime
import subprocess

# Derive the root of this package
package_root = Path.cwd().resolve().parents[2]

# Include this pakage
HaMaGeoLib_DIR = "/home/lochy/ASPECT_PROJECT/HaMaGeoLib"
if os.path.abspath(HaMaGeoLib_DIR) not in sys.path:
    sys.path.append(os.path.abspath(HaMaGeoLib_DIR))
from hamageolib.utils.exception_handler import my_assert
import hamageolib.utils.plot_helper as plot_helper

# Working directories
local_Collision_dir = "/mnt/lochy/ASPECT_DATA/Collision0" # data directory
# remote_ThDSubduction_dir = "peloton:/group/billengrp-mpi-io/lochy/ThDSubduction"
assert(os.path.isdir(local_Collision_dir))

# py_temp file and temperature results directory
py_temp_dir = os.path.join(HaMaGeoLib_DIR, "py_temp_files")
RESULT_DIR = os.path.join(HaMaGeoLib_DIR, 'results')
os.makedirs(py_temp_dir, exist_ok=True) # Ensure the directory exists

# paraview scripts directory
SCRIPT_DIR = os.path.join(HaMaGeoLib_DIR, "scripts")

today_date = datetime.datetime.today().strftime("%Y-%m-%d") # Get today's date in YYYY-MM-DD format
py_temp_file = os.path.join(py_temp_dir, f"py_temp_{today_date}.sh")

if not os.path.exists(py_temp_file):
    bash_header = """#!/bin/bash
# =====================================================
# Script: py_temp.sh
# Generated on: {date}
# Description: Temporary Bash script created by Python
# =====================================================

""".format(date=today_date)
    with open(py_temp_file, "w") as f:
        f.write(bash_header)

print(f"File ensured at: {py_temp_file}")

# Pre Process

## Continent Strength Profile

In [ ]:
## todo_ct
is_continent_strength = True

### Set up Nnalysis of the Continental Profile

Here we:
1. read in profile of P, T of the coninental.
2. parse the parameter file and read the rheological parameters

In [ ]:

if is_continent_strength:

    from gdmate.aspect.table import DepthAverageTable
    from gdmate.aspect.io import parse_parameters_to_dict, parse_composition_entry, parse_entry_as_list

    # Hard code the depth of layers, this is set up as the initial composition in the continental_extension.prm
    depth_levels = [20e3, 40e3]  # crust_upper, crust_lower, mantle_lithosphere
    
    # Read the T, P profile from running the continental extension cookbook
    # extract data at time 0.0
    da_file = package_root/"hamageolib/research/haoyuan_collision0/files/continental_extensiion/depth_average.txt"
    my_assert(da_file.is_file(), FileNotFoundError, "%s doesn't exist" % str(da_file))

    my_table = DepthAverageTable(da_file)

    profile_T = my_table.profile(time=0.0, field="temperature")
    profile_P = my_table.profile(time=0.0, field="adiabatic_pressure")

    # Read the prm file from the original continental extension cookbook
    # Run parser
    prm_path = package_root/"hamageolib/research/haoyuan_collision0/files/continental_extensiion/continental_extension.prm"
    assert prm_path.exists(), f"Test file not found: {prm_path}"

    with open(prm_path, "r") as fin:
        params_dict = parse_parameters_to_dict(fin)

    # Parse the compositions
    compositional_field_names = parse_entry_as_list(params_dict["Compositional fields"]["Names of fields"])
    compositional_field_types = parse_entry_as_list(params_dict["Compositional fields"]["Types of fields"])

    chemical_composition_list = []
    for i, type in enumerate(compositional_field_types):
        if type == "chemical composition":
            chemical_composition_list.append(compositional_field_names[i])

    # Parse the rheological parameters
    material_dict = params_dict["Material model"]["Visco Plastic"]

    disl_A_dict = parse_composition_entry(material_dict["Prefactors for dislocation creep"])
    disl_n_dict = parse_composition_entry(material_dict["Stress exponents for dislocation creep"])
    disl_E_dict = parse_composition_entry(material_dict["Activation energies for dislocation creep"])
    disl_V_dict = parse_composition_entry(material_dict["Activation volumes for dislocation creep"])

    dislocation_aspect = {}
    for composition in chemical_composition_list:
        dislocation_aspect_comp = {
            'A': float(disl_A_dict[composition]), 
            'd': float(material_dict.get("Grain size", 1e-3)), 
            'n': float(disl_n_dict[composition]), 
            'm': 0.0, 
            'E': float(disl_E_dict[composition]), 
            'V': float(disl_V_dict[composition])
        }
        dislocation_aspect[composition] = dislocation_aspect_comp
    
    friction_angle = float(material_dict["Angles of internal friction"])
    cohesion = float(material_dict["Cohesions"])

    # print parse results
    print("chemical_composition_list: ")
    print(chemical_composition_list)
    print("dislocation_aspect: ")
    print(dislocation_aspect)

### Construct the Strength Profile

Here we:
1. Assign a depth array and interpolate the P, T profile from on we read in the last step.
2. Assume constant strain rate and compute the viscosity.
3. Use composite regime to combine different rheologic mechanisms.

In [ ]:
if is_continent_strength:

    from hamageolib.research.haoyuan_2d_subduction.legacy_tools import CoulumbYielding, CreepRheologyInAspectViscoPlastic

    # compute at this constant strain rate
    strain_rate = 1e-14

    # yield stress of brittle yielding
    depths = np.arange(0, 101e3, 1e3)
    Ts = np.interp(depths, profile_T["depth"].to_numpy(), profile_T["temperature"].to_numpy())
    Ps = np.interp(depths, profile_P["depth"].to_numpy(), profile_P["adiabatic_pressure"].to_numpy())

    friction = np.tan(friction_angle*np.pi/180.0)
    taus_brittle = CoulumbYielding(Ps, cohesion, friction)
    eta_brittle = taus_brittle / 2.0 / strain_rate

    # dislocation creep
    # use mask to compute for each compositions
    eta_dislocation = np.zeros(depths.size)

    composition = "crust_upper"
    mask_c = (depths <= depth_levels[0])
    eta_dislocation[mask_c] = CreepRheologyInAspectViscoPlastic(dislocation_aspect[composition], strain_rate, Ps[mask_c], Ts[mask_c])
    
    composition = "crust_lower"
    mask_c = ((depths > depth_levels[0]) & (depths <= depth_levels[1]))
    eta_dislocation[mask_c] = CreepRheologyInAspectViscoPlastic(dislocation_aspect[composition], strain_rate, Ps[mask_c], Ts[mask_c])
    
    composition = "mantle_lithosphere"
    mask_c = (depths > depth_levels[1])
    eta_dislocation[mask_c] = CreepRheologyInAspectViscoPlastic(dislocation_aspect[composition], strain_rate, Ps[mask_c], Ts[mask_c])

    # placeholder for diffusion creep
    eta_diffusion = np.full(depths.shape, np.inf)

    # placeholder for Peierls creep
    eta_peierls = np.full(depths.shape, np.inf)
    taus_peierls = np.full(depths.shape, np.inf)

    # combine diffusion and dislocation
    eta_dfds = 1.0 / (1.0 / eta_diffusion + 1.0 / eta_dislocation)
    taus_dfds = 2 * strain_rate * eta_dfds

    # combine deformation mechanisms follow DP-yielding
    eta = 1.0 / (1.0/eta_brittle + 1.0/eta_peierls + 1.0/eta_dfds)
    eta1 = np.minimum(eta_peierls, eta_dfds)  # minimum
    eta1 = np.minimum(eta_brittle, eta1)
    eta2 = np.minimum(eta_brittle, 1.0 / (1.0 / eta_peierls + 1.0/eta_dfds)) # DP yield
    eta3 = np.minimum(eta_brittle, 1.0/(1.0 / np.minimum(eta_peierls, eta_dislocation) + 1.0 / eta_diffusion)) # competing Peierls and Dislocation
    eta_nopc = np.minimum(eta_brittle, eta_dfds)
    taus = 2 * strain_rate * eta
    taus1 = 2 * strain_rate * eta1
    taus2 = 2 * strain_rate * eta2
    taus3 = 2 * strain_rate * eta3
    taus_nopc = 2 * strain_rate * eta_nopc

### Plot results

In [ ]:
# todo_ct
print(taus_brittle)

In [ ]:
if is_continent_strength:

    from matplotlib import gridspec, rcdefaults 
    from matplotlib.ticker import MultipleLocator
    from hamageolib.utils.plot_helper import scale_matplotlib_params

    # Retrieve the default color cycle
    default_colors = [color['color'] for color in plt.rcParams['axes.prop_cycle']]

    # Scaling parameters for plots.
    scaling_factor = 1.6  # General scaling factor for the plot size.
    font_scaling_multiplier = 1.5  # Extra scaling for fonts.
    legend_font_scaling_multiplier = 0.5  # Scaling for legend fonts.
    line_width_scaling_multiplier = 2.0  # Extra scaling for line widths
    n_minor_ticks = 4  # number of minor ticks between two major ones


    # Scale matplotlib parameters based on specified factors.
    scale_matplotlib_params(
        scaling_factor, 
        font_scaling_multiplier=font_scaling_multiplier,
        legend_font_scaling_multiplier=legend_font_scaling_multiplier,
        line_width_scaling_multiplier=line_width_scaling_multiplier
    )

    # Update font settings for compatibility with publishing tools like Illustrator.
    plt.rcParams.update({
        'font.family': 'Times New Roman',
        'pdf.fonttype': 42,
        'ps.fonttype': 42
    })

    # plot
    # 1. shear stress vs depth
    fig = plt.figure(tight_layout=True, figsize=(12, 10))
    gs = gridspec.GridSpec(2, 2)

    ax = fig.add_subplot(gs[0, 0])
    ax.plot(taus2/1e6, depths/1e3, label="Composite, DP yield")
    ax.plot(taus_nopc/1e6, depths/1e3, label="Composite (no peierls)")
    ax.plot(taus_brittle/1e6, depths/1e3, 'b--', label="Brittle")
    ax.plot(taus_peierls, depths/1e3, 'c--', label="Peierls") # peierls is MPa
    ax.plot(taus_dfds/1e6, depths/1e3, 'g--', label="Diff-Disl")

    ax.set_xlabel("Stress (MPa)")
    ax.set_xlim([0, 1000.0])
    x_tick_interval = 250.0
    ax.xaxis.set_major_locator(MultipleLocator(x_tick_interval))
    ax.xaxis.set_minor_locator(MultipleLocator(x_tick_interval/(n_minor_ticks+1)))

    ax.set_ylabel("Depth (km)")
    ax.set_ylim([0, 100])
    ax.invert_yaxis()
    y_tick_interval = 25.0
    ax.yaxis.set_major_locator(MultipleLocator(y_tick_interval))
    ax.yaxis.set_minor_locator(MultipleLocator(y_tick_interval/(n_minor_ticks+1)))

    ax.grid()
    ax.legend()

    # 2. viscosity vs depth
    ax = fig.add_subplot(gs[0, 1])

    ax.plot(np.log10(eta2), depths/1e3, label="Composite, DP yield")
    ax.plot(np.log10(eta_brittle), depths/1e3, 'b--', label="Brittle")
    ax.plot(np.log10(eta_peierls), depths/1e3, 'c--', label="Peierls")
    ax.plot(np.log10(eta_dfds), depths/1e3, 'g--', label="Diff-Disl")

    ax.set_xlabel("log10(Viscosity) (Pa s)")
    ax.set_xlim([18.0, 24.0])
    x_tick_interval = 1.0
    ax.xaxis.set_major_locator(MultipleLocator(x_tick_interval))
    ax.xaxis.set_minor_locator(MultipleLocator(x_tick_interval/(n_minor_ticks+1)))

    ax.set_ylabel("Depth (km)")
    ax.set_ylim([0, 100])
    ax.invert_yaxis()
    y_tick_interval = 25.0
    ax.yaxis.set_major_locator(MultipleLocator(y_tick_interval))
    ax.yaxis.set_minor_locator(MultipleLocator(y_tick_interval/(n_minor_ticks+1)))

    ax.grid()
    ax.set_title("strain rate = %.2e" % strain_rate)

    # 3. shear stress vs depth
    ax = fig.add_subplot(gs[1, 0])

    ax.plot(taus2/1e6, depths/1e3, label="Composite, DP yield")
    ax.plot(taus/1e6, depths/1e3, label="Composite, SL yield")
    ax.plot(taus_nopc/1e6, depths/1e3, label="Composite (no peierls)")
    ax.plot(taus1/1e6, depths/1e3, label="Minimization")
    ax.plot(taus3/1e6, depths/1e3, label="Min(Peierls, dislocation)")

    ax.set_xlabel("Stress (MPa)")
    ax.set_xlim([0, 1000.0])
    x_tick_interval = 250.0
    ax.xaxis.set_major_locator(MultipleLocator(x_tick_interval))
    ax.xaxis.set_minor_locator(MultipleLocator(x_tick_interval/(n_minor_ticks+1)))

    ax.set_ylabel("Depth (km)")
    ax.set_ylim([0, 100])
    ax.invert_yaxis()
    y_tick_interval = 25.0
    ax.yaxis.set_major_locator(MultipleLocator(y_tick_interval))
    ax.yaxis.set_minor_locator(MultipleLocator(y_tick_interval/(n_minor_ticks+1)))

    ax.grid()
    ax.legend()

    rcdefaults()

# Create Cases

In [ ]:
# record of configurations
# thicker weak layer including "gabbro" with a higher viscosity
            #   "weak_layer_compositions": ["MORB", "sediment", "gabbro"], # weak layer
            #   "weak_layer_viscosity": 1e20, # weak layer


In [ ]:
is_create_new_case = False

if is_create_new_case:

    import json
    import copy
    from gdmate.aspect.config_engine import RuleEngine
    from gdmate.aspect.builtin_rules import CasePathRule, InitialStepRule
    from gdmate.aspect.io import parse_parameters_to_dict, save_parameters_from_dict
    from hamageolib.research.haoyuan_collision0.config import CaseNameFromVariables, GeometryRule, PostProcessorRule, RemoveFluidRule, RemovePeridotiteRule,\
        RheologyRule, WeakLayerRule, SlabRule, SolverRule, PrescribConditionRule
    
    root_dir = Path("/mnt/lochy/ASPECT_DATA/Collision0/collision_setup")
    
    template_prm = package_root/"hamageolib/research/haoyuan_collision0/files/01112026/post_compressible_test.prm"
    template_wb = package_root/"hamageolib/research/haoyuan_collision0/files/01112026/original.wb"

    # Make the root dir
    root_dir.mkdir(exist_ok=True) 
    
    # Initiate rule engine
    ruleEngine = RuleEngine([PostProcessorRule(), CasePathRule(), RemoveFluidRule(), RemovePeridotiteRule(), 
                             SlabRule(), GeometryRule(), RheologyRule(), WeakLayerRule(), PrescribConditionRule(), 
                             SolverRule()])
    ruleEngineInitalStep = RuleEngine([InitialStepRule()])
    
    # Read prm and wb templates
    assert(template_prm.is_file())
    with template_prm.open('r') as fin:
        prm_dict = parse_parameters_to_dict(fin, format_entry=True)
    assert(template_wb.is_file())
    with template_wb.open('r') as fin:
        wb_dict = json.load(fin)
    
    # debug the formate_entry option
    foo = prm_dict["Compositional fields"]["Mapped particle properties"]
    
    # Apply case configuration rules
    config = {"use_my_setup_of_postprocess": True,
            #   "remove_fluid": True,  # remove fluid
            #   "remove_fluid_compositions": ["porosity", "bound_fluid"], # remove fluid
              "remove_peridotite": True, # remove peridotite
              "domain_depth": 1000e3, "repetition_length": 500e3, "use_isosurfaces": True, # geometry
              "use_my_setup_of_rheology": True, # rheology
              "viscosity_range": [2.5e19, 2.5e23], # rheology
              "use_safer_options": True, # rheology, set adiabatic pressure and cutoff negative temperature
            #   "weak_layer_compositions": ["MORB", "sediment"], # weak layer, normal setting
              "weak_layer_compositions": ["MORB", "sediment", "gabbro"], # weak layer, setting thicker weak layers
              "force_weak_layer_max_refinement": True, # weak layer
              "slab_layer_compositions": ["sediment", "MORB", "gabbro"], # slab
              "slab_layer_depths": [0.0, 4e3, 7.5e3, 15e3], # slab & remove peridotite
              "stokes_solver_type": "block GMG",
              "skip_expensive_stokes": True, # solver
              "max_nonlinear_iterations": 40, # solver
              "linear_solver_tolerance": 5e-3, # solver
              "number_of_cheap_Stokes_solver_steps": 60, # solver
              "GMRES_solver_restart_length": 100, # solver
              "prescribe_subducting_plate_velocity": True, # prescribed condition
              "prescribe_subducting_plate_velocity_region_method": "relative_to_hinge", # prescribed condition
              "prescribe_subducting_plate_velocity_velocity_method":"spreading_velocity", # prescribed condition
              "prescribe_subducting_plate_velocity_hinge_relative_distance": 1000e3, # prescribed condition
              "prescribe_subducting_plate_velocity_length": 500e3, # prescribed condition
              "prescribe_subducting_plate_velocity_depth_range": [20e3, 40e3], # prescribed condition
              "convergence_rate": 0.05, # prescribed condition
              }

    contexts, documentation = ruleEngine.apply_all(config, prm_dict, wb_dict)

    # create case directory based on name patterns
    # check existing ones and ask user permission to proceed
    variables = ruleEngine.get_required_variables(config)
    case_name = CaseNameFromVariables(variables)
    case_dir = root_dir/case_name

    if case_dir.is_dir():
        ans = input("Director %s already exist, overwrite [y/n]?" % str(case_dir))
        if ans == "y":
            print("Going to overwrite the pre-existing case.")
        else:
            print("Cell execution stopped.")
            raise SystemExit
        
    case_dir.mkdir(exist_ok=True)
    case_prm = case_dir/"case.prm"
    case_prm_initial = case_dir/"case_ini.prm"
    case_wb = case_dir/"case.wb"
    
    # write case prm file
    with case_prm.open('w') as fout:
        save_parameters_from_dict(fout, prm_dict)
    with case_wb.open('w') as fout:
        json.dump(wb_dict, fout, indent=4)
    print("Saved prm file %s" % str(case_prm))
    print("Saved wb file %s" % str(case_wb))
    
    # write the initial step prm file
    config_initial = {"n_trivial_step": 1}
    prm_dict_initial = copy.deepcopy(prm_dict)
    _, _  = ruleEngineInitalStep.apply_all(config_initial, prm_dict_initial, wb_dict)
    with case_prm_initial.open('w') as fout:
        save_parameters_from_dict(fout, prm_dict_initial)
    print("Saved prm file %s" % str(case_prm_initial))

    # save documentation
    doc_dir = case_dir/"doc"
    if not doc_dir.is_dir():
        doc_dir.mkdir()
    
    full_doc_path = doc_dir/"full_doc.md"
    doc_md = ruleEngine.render_docs_markdown(documentation)
    with full_doc_path.open("w") as fout:
        fout.write(doc_md)
    print("Saved full documentation %s" % str(full_doc_path))
    
    full_table_path = doc_dir/"full_table.md"
    table_md = ruleEngine.render_docs_table(documentation)
    with full_table_path.open("w") as fout:
        fout.write(table_md)
    print("Saved full table %s" % str(full_table_path))

    full_json_path = doc_dir/"full.json"
    with full_json_path.open("w") as fout:
        json.dump(documentation, fout, indent=2)
    print("Saved full json %s" % str(full_json_path))



# Post Process

We now begin processing the simulation cases.
The first step is to enumerate the case directories and confirm that all specified paths exist on the filesystem.

Options:

* do_post_process - control the run of the following code blocks


In [ ]:
do_post_process = True

In [ ]:
if do_post_process:
    # placeholder for a 3-D case
    case_name = None 

    # Normally, I don't need to specify name of the file, they are case.prm and case.wb be default
    prm_basename_2d = "case.prm"; wb_basename_2d = "case.wb"; output_directory="output"
    # Initial test from Fritz
    # case_name_2d = "collision_test/Fritz_test0"; prm_basename_2d = "init_compressible_test.prm"; wb_basename_2d = "original.wb"; output_directory="isentropic_adiabat_new_boundary"

    # Test with shallow domain
    case_name_2d = "collision_setup/D1000_WLCG_WLV1.0e+20"
    
    # Test with shallow domain, removed fluid
    # case_name_2d = "collision_test/test0"; prm_basename_2d = "case_ini.prm"

    local_dir = None; local_dir_2d = None
    if case_name is not None:
        local_dir = os.path.join(local_Collision_dir, case_name)
        assert(os.path.isdir(local_dir))
        print("local_dir:\n\t", local_dir)
        subprocess.run(['mkdir', '-p', '%s/img/pv_outputs' % local_dir])
    if case_name_2d is not None:
        local_dir_2d = os.path.join(local_Collision_dir, case_name_2d)
        assert(os.path.isdir(local_dir_2d))
        print("local_dir_2d:\n\t", local_dir_2d)
        subprocess.run(['mkdir', '-p', '%s/img/pv_outputs' % local_dir_2d])

## Generate Paraview script

We apply a combined workflow with **ParaView**.  
In this notebook, we generate the processing script.  
We then use ParaView to refine styling and export the final plots.

options

- `is_prepare_for_plot` — Generate the **ParaView** Python script (no data pre-processing).  
- `is_process_pyvista_for_plot` — Pre-process results with **PyVista** before exporting the ParaView script, then proceed in ParaView.


In [ ]:
is_prepare_for_plot = True
is_process_pyvista_for_plot = False

In [ ]:
if do_post_process and is_prepare_for_plot:


    from hamageolib.research.haoyuan_collision0.case_options import CASE_OPTIONS_TWOD
    from hamageolib.research.haoyuan_collision0.post_process import ProcessVtuFileTwoDStep, GenerateParaviewScript

    # check again case directory exists    
    assert(local_dir_2d is not None)

    # options 
    graphical_step = 0

    # original script
    ofile_list = ["collision0.py"]; require_base=True

    # automatically read case configurations
    Case_Options_2d = CASE_OPTIONS_TWOD(local_dir_2d, case_file=prm_basename_2d, wb_basename=wb_basename_2d, output_directory=output_directory)
    Case_Options_2d.Interpret(step=graphical_step)
    Case_Options_2d.SummaryCaseVtuStep(os.path.join(local_dir_2d, "summary.csv"))

    # the index of the vtu file depends on whether all the intial adaptive steps are output
    pvtu_step = Case_Options_2d.get_pvtu_step(graphical_step)

    ProcessVtuFileTwoDStep(local_dir_2d, pvtu_step, Case_Options_2d)

    GenerateParaviewScript(local_dir_2d, Case_Options_2d, ofile_list, require_base=require_base)

## Automize figure finalization

The figures generated from the previous **ParaView** script can be finalized using the following code block.
We would use a "frame" figure generated in Adobe illustrator and overlay that on the ParaView plot.

Options:
- `file_name` — Name of the plot to finalize.
- `_time` — Model time associated with the plot.


In [ ]:
# todo_collision
finalize_visual = False

if do_post_process and finalize_visual:
    
    from hamageolib.research.haoyuan_collision0.post_process import finalize_visualization_2d_11022025 

    # options for file_name and time
    file_name = "full_domain_viscosity" # "full_domain_viscosity", "full_model_temperature"
    _time = 2.9014e+06

    frame_png_file_with_ticks = "/home/lochy/Documents/papers/documented_files/collision/full_domain_frame_trans-01.png"
    output_image_file = finalize_visualization_2d_11022025(local_dir_2d, file_name, _time, frame_png_file_with_ticks, add_time=False)


## Animation

### 2-d case, basic

First generate the plotting scripts, then run them in the terminal. Assemble the final animation only after all figures have been generated. In practice, this section may need to be executed multiple times.


In [ ]:
animate_2d_case_basic = False
debug_step0_animate_2d_case_basic = True
generate_paraview_scripts_for_animate_2d_case_basic = True

if animate_2d_case_basic:
    
    from hamageolib.research.haoyuan_collision0.case_options import CASE_OPTIONS_TWOD
    from hamageolib.research.haoyuan_collision0.post_process import ProcessVtuFileTwoDStep, GenerateParaviewScript

    # Assign a time interval for animation
    time_interval = 0.5e6
    animation_name= "ani_basic"
    max_depth = "1500"

    # Apply case options
    Case_Options_2d = CASE_OPTIONS_TWOD(local_dir_2d, case_file=prm_basename_2d, wb_basename=wb_basename_2d, output_directory=output_directory)
    Case_Options_2d.Interpret()
    # Case_Options_2d.SummaryCaseVtuStep(os.path.join(local_dir_2d, "summary.csv"))
    # Case_Options_2d.SummaryCaseVtuStepExport(os.path.join(local_dir_2d, "summary.csv"))
    resampled_df = Case_Options_2d.resample_visualization_df(time_interval)
    graphical_steps = resampled_df["Vtu step"].values

#### Generate **ParaView** scripts stepwise

Optional: generate stepwise **ParaView** scripts controlled by `generate_paraview_scripts_for_animate_2d_case_basic` option.
After this step, run `py_temp.py` script in a terminal to produce the visualizations.

In [ ]:
if animate_2d_case_basic and generate_paraview_scripts_for_animate_2d_case_basic: 

    # Open py_temp_file for output
    fout = open(py_temp_file, 'w')
    assert(fout)
    fout.write("#!/bin/bash\n")

    # Run stepwise
    print("Start generating paraview scripts")
    for i, _time in enumerate(resampled_df["Time"].values):

        # debug run step 0
        if debug_step0_animate_2d_case_basic:
            if i > 0:
                break

        # Stepwise configurations 
        _time = float(_time)
        time_rounded = round(_time / float(resampled_df.attrs["Time between graphical output"]))\
              * float(resampled_df.attrs["Time between graphical output"])
        step = int(graphical_steps[i])
        print("\tGenerating paraview scripts for step = %d, time = %.4e" % (step, time_rounded))

        # Assign the script to use
        py_script = "collision0.py"

        # Make the directory to hold the scripts
        ps_dir = os.path.join(local_dir_2d, 'paraview_scripts')
        if not os.path.isdir(ps_dir):
            os.mkdir(ps_dir) 
        odir = os.path.join(ps_dir, "stepwise")
        if not os.path.isdir(odir):
            os.mkdir(odir)

        # Apply stepwise configuration
        Case_Options_2d.options['ANIMATION'] = "True"
        Case_Options_2d.options['GRAPHICAL_STEPS'] = [step]
        Case_Options_2d.options['GRAPHICAL_TIMES'] = [time_rounded]

        ofile = os.path.join(odir, 'slab_%d.py' % (step))
        paraview_script = os.path.join(SCRIPT_DIR, 'paraview_scripts', 'Collision0', py_script)
        paraview_script_base = os.path.join(SCRIPT_DIR, 'paraview_scripts', 'base.py')
        Case_Options_2d.read_contents(paraview_script_base, paraview_script)

        # Save script
        Case_Options_2d.substitute()
        Case_Options_2d.save(ofile)

        # Write to py_temp file
        fout.write("pvpython %s\n" % ofile)

    # Finish writting to py_temp file
    fout.close()
    subprocess.run(["chmod", "+x", py_temp_file])
    print("saved file: %s" % py_temp_file)

#### Finalize plot from **ParaView**

Ensure the previous step has completed. Use the following cell to finalize plots generated by **ParaView**.

In [ ]:
need_framing = False

if animate_2d_case_basic:
    
    from hamageolib.research.haoyuan_collision0.post_process import finalize_visualization_2d_11022025
    from hamageolib.utils.plot_helper import extract_image_by_size

    # file types
    file_name_list = ["full_domain_viscosity", "full_domain_temperature", "full_domain_density"]
    prep_dir = os.path.join(local_dir_2d, "img", "prep")
    if not os.path.isdir(prep_dir):
        os.mkdir(prep_dir)

    print("Start Finalizing Plots")
    for i, _time in enumerate(resampled_df["Time"].values):

        # debug run step 0
        if debug_step0_animate_2d_case_basic:
            if i > 0:
                break

        # Stepwise configurations 
        _time = resampled_df["Time"].values[i]
        time_rounded = round(_time / float(resampled_df.attrs["Time between graphical output"]))\
              * float(resampled_df.attrs["Time between graphical output"])
        step = graphical_steps[i]
        print("\tFinalizing plots for step = %d, time = %.4e" % (step, time_rounded))

        for file_name in file_name_list:
            if need_framing:
                # framing with plot with preprepared frames
                frame_png_file_with_ticks = "/home/lochy/Documents/papers/documented_files/collision/full_domain_frame_trans_frame-01.png"
                output_image_file = finalize_visualization_2d_11022025(local_dir_2d, file_name, time_rounded, frame_png_file_with_ticks, add_time=False)
            else:
                # if not only check file existence
                eps_file = os.path.join(local_dir_2d, "img", "pv_outputs", "%s_t%.4e.eps" % (file_name, time_rounded))
                pdf_file = os.path.join(local_dir_2d, "img", "pv_outputs", "%s_t%.4e.pdf" % (file_name, time_rounded))


                if (not os.path.isfile(eps_file)) and (not os.path.isfile(pdf_file)):
                    raise FileNotFoundError(f"Neither the EPS nor pdf exists: {eps_file}, {pdf_file}")
                
                # if os.path.isfile(pdf_file):
                #     target_size = (1350, 704)  # Desired image dimensions in pixels
                #     crop_box = (0, 0, 1000, 1000)  # Optional crop box
                #     full_image_path = extract_image_by_size(pdf_file, target_size, prep_dir, crop_box)
                #     copy(full_image_path, os.path.join(prep_dir, "%s_t%.4e.png" % (file_name, time_rounded)))
                #     print("Saved file as %s" % os.path.join(prep_dir, "%s_t%.4e.png" % (file_name, time_rounded)))
                copy(os.path.join(local_dir_2d, "img", "pv_outputs", "%s_t%.4e.png" % (file_name, time_rounded)),\
                    os.path.join(prep_dir, "%s_t%.4e.png" % (file_name, time_rounded)))
                print("Saved file as %s" % os.path.join(prep_dir, "%s_t%.4e.png" % (file_name, time_rounded)))


#### Assemble and make animation

Using the generated figures (e.g., linear plots and finalized figures from **ParaView**):

1. Assemble them stepwise and export one combined figure per step.
2. Create an `.avi` animation from the combined figures.


In [ ]:
if animate_2d_case_basic:

    print("Start making animation")

    # Load modules
    from hamageolib.research.haoyuan_2d_subduction.workflow_scripts import create_avi_from_images

    # Initiation
    ani_file_paths = [] # path of the figures

    # Loop the steps to get job done
    for i, _time in enumerate(resampled_df["Time"].values):

        # debug run step 0
        if debug_step0_animate_2d_case_basic:
            if i > 0:
                break

        # do this if there is error at the last step
        if i == resampled_df["Time"].values.size - 1:
            break

        # Stepwise configurations 
        _time = resampled_df["Time"].values[i]
        time_rounded = round(_time / float(resampled_df.attrs["Time between graphical output"]))\
              * float(resampled_df.attrs["Time between graphical output"])
        step = graphical_steps[i]
        print("\tAssembling plots for step = %d, time = %.4e" % (step, time_rounded))

        # File paths
        image_files = []; image_positions=[]; cropping_regions=[]; image_scale_factors=[]

        # viscosity slice
        file_path_0 = os.path.join(prep_dir, "%s_t%.4e.png" % (file_name_list[0], time_rounded))
        assert(os.path.isfile(file_path_0))
        image_files.append(file_path_0)
        image_positions.append((0, 100)) 
        cropping_regions.append(None)
        image_scale_factors.append(1.5)

        if need_framing:
            # viscosity colorbar
            file_path_0_c = "/home/lochy/Documents/papers/documented_files/collision/color_viscosity_18_24-01.png"
            assert(os.path.isfile(file_path_0_c))
            image_files.append(file_path_0_c)
            image_positions.append((300, 650)) 
            cropping_regions.append(None)
            image_scale_factors.append(1.5)

        # temperature slice
        file_path_0 = os.path.join(prep_dir, "%s_t%.4e.png" % (file_name_list[1], time_rounded))
        assert(os.path.isfile(file_path_0))
        image_files.append(file_path_0)
        image_positions.append((2000, 100)) 
        cropping_regions.append(None)
        image_scale_factors.append(1.5)
        
        if need_framing:
            # temperature colorbar
            file_path_2_c = "/home/lochy/Documents/papers/documented_files/collision/color_temperature_0_2000.png"
            assert(os.path.isfile(file_path_2_c))
            image_files.append(file_path_2_c)
            image_positions.append((1900, 700)) 
            cropping_regions.append(None)
            image_scale_factors.append(1.5)

        # density slice
        file_path_0 = os.path.join(prep_dir, "%s_t%.4e.png" % (file_name_list[2], time_rounded))
        assert(os.path.isfile(file_path_0))
        image_files.append(file_path_0)
        image_positions.append((0, 1200)) 
        cropping_regions.append(None)
        image_scale_factors.append(1.5)
        
        if need_framing:
            # density colorbar
            file_path_2_c = "/home/lochy/Documents/papers/documented_files/collision/color_density_3000_4000-01.png"
            assert(os.path.isfile(file_path_2_c))
            image_files.append(file_path_2_c)
            image_positions.append((300, 1450)) 
            cropping_regions.append(None)
            image_scale_factors.append(1.5)

        # Combine images
        output_image_file = os.path.join(prep_dir, "%s_t%.4e.png" % (animation_name, _time))
        # Remove existing output image to ensure a clean overlay
        if os.path.isfile(output_image_file):
            os.remove(output_image_file)
        # Call overlay function
        plot_helper.overlay_images_on_blank_canvas(
            canvas_size=(4200, 2500),  # Size of the blank canvas in pixels (width, height)
            image_files=image_files,  # List of image file paths to overlay
            image_positions=image_positions,  # Positions of each image on the canvas
            cropping_regions=cropping_regions,  # Optional cropping regions for the images
            image_scale_factors=image_scale_factors,  # Scaling factors for resizing the images
            output_image_file=output_image_file  # Path to save the final combined image
        )

        # Add time stamp
        text = "t = %.1f Ma" % (time_rounded / 1e6)  # Replace with the text you want to add
        position = (25, 0)  # Replace with the desired text position (x, y)
        font_path = "/usr/share/fonts/truetype/msttcorefonts/times.ttf"  # Path to Times New Roman font
        font_size = 56

        plot_helper.add_text_to_image(output_image_file, output_image_file, text, position, font_path, font_size)

        ani_file_paths.append(output_image_file)

    # Generate animation
    if not debug_step0_animate_2d_case_basic:
        ani_dir = os.path.join(local_dir_2d, "img", "animation")
        if not os.path.isdir(ani_dir):
            os.mkdir(ani_dir)
        output_file = os.path.join(local_dir_2d, "img", "animation", "%s.avi" % animation_name)
        create_avi_from_images(ani_file_paths, output_file, 1)